In [11]:
!curl -sSL  https://install.python-poetry.org | POETRY_HOME=/etc/poetry python - && PATH="/etc/poetry/bin:$PATH" && poetry completions bash >> ~/.bash_completion && poetry config virtualenvs.create false && cd .. && poetry install && poetry build

Retrieving Poetry metadata

The latest version (1.8.3) is already installed.

Poetry could not find a pyproject.toml file in /databricks or its parents

In [12]:
!pip install ../dist/*.whl

CalledProcessError : Command 'pip --disable-pip-version-check install ../dist/*.whl' returned non-zero exit status 1.

Error: [0;31m---------------------------------------------------------------------------[0m
[0;31mCalledProcessError[0m                        Traceback (most recent call last)
File [0;32m~/.ipykernel/4410/command--1-2581027883:1[0m
[0;32m----> 1[0m get_ipython()[38;5;241m.[39mrun_line_magic([38;5;124m'[39m[38;5;124mpip[39m[38;5;124m'[39m, [38;5;124m'[39m[38;5;124minstall ../dist/*.whl[39m[38;5;124m'[39m)

File [0;32m/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2432[0m, in [0;36mInteractiveShell.run_line_magic[0;34m(self, magic_name, line, _stack_depth)[0m
[1;32m   2430[0m     kwargs[[38;5;124m'[39m[38;5;124mlocal_ns[39m[38;5;124m'[39m] [38;5;241m=[39m [38;5;28mself[39m[38;5;241m.[39mget_local_scope(stack_depth)
[1;32m   2431[0m [38;5;28;01mwith[39;00m [38;5;28mself[39m[38;5;241m.[39mbuiltin_trap:
[0;32m-> 2432[0m     result [38;5;241m=[39m fn([38;5;241m*[39margs, [38;5;241m*[39m[38;5;241m*[3

In [3]:
dbutils.library.restartPython()

In [4]:
import torch
from torch import Tensor
from torch.utils.data import Dataset


class RandomDataset(Dataset):
    def __init__(self, size: int, num_samples: int) -> None:
        self.len = num_samples
        self.data = torch.randn(num_samples, size)

    def __getitem__(self, index: int) -> Tensor:
        return self.data[index]

    def __len__(self) -> int:
        return self.len

In [5]:
import lightning as L
import torch
from torch.utils.data import DataLoader


class RandomDataModule(L.LightningDataModule):
    def __init__(self, size: int = 32, num_samples: int = 10000, batch_size: int = 32) -> None:
        """The Random data module.

        Args:
            size: The tensor size.
            num_samples: The number of samples.
            batch_size: The batch size.
        """
        super().__init__()
        self.size = size
        self.num_samples = num_samples
        self.batch_size = batch_size

    def setup(self, stage: str | None = None) -> None:
        """Setup the data module per stage.

        Args:
            stage: The training stage.
        """
        self.data_test = RandomDataset(self.size, self.num_samples)
        self.data_train = RandomDataset(self.size, self.num_samples)
        self.data_val = RandomDataset(self.size, self.num_samples)

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.data_train, batch_size=self.batch_size)

    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.data_val, batch_size=self.batch_size)

    def test_dataloader(self) -> DataLoader:
        return DataLoader(self.data_test, batch_size=self.batch_size)

Unexpected internal error when monkey patching `PreTrainedModel.from_pretrained`: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'assume_constant_result' from partially initialized module 'torch._dynamo' (most likely due to a circular import) (/databricks/python/lib/python3.11/site-packages/torch/_dynamo/__init__.py)
Unexpected internal error when monkey patching `Trainer.train`: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
cannot import name 'assume_constant_result' from partially initialized module 'torch._dynamo' (most likely due to a circular import) (/databricks/python/lib/python3.11/site-packages/torch/_dynamo/__init

In [6]:
from typing import Any

import lightning as L
import torch


class BoringModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Linear(32, 2)

        self.training_step_outputs = []
        self.validation_step_outputs = []

    def forward(self, x):
        return self.layer(x)

    def loss(self, batch, prediction):
        # An arbitrary loss to have a loss that updates the model weights during `Trainer.fit` calls
        return torch.nn.functional.mse_loss(prediction, torch.ones_like(prediction))

    def training_step(self, batch, batch_idx) -> dict[str, Any]:
        output = self(batch)
        loss = self.loss(batch, output)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx) -> None:
        output = self(batch)
        loss = self.loss(batch, output)
        self.log("valid_loss", loss)

    def test_step(self, batch, batch_idx) -> None:
        output = self(batch)
        loss = self.loss(batch, output)
        self.log("test_loss", loss)

    # def on_train_epoch_end(self, outputs) -> Any:
    #     return torch.stack([x["loss"] for x in outputs]).mean()

    # def on_validation_epoch_end(self, outputs) -> Any:
    #     return torch.stack([x["x"] for x in outputs]).mean()

    # def on_test_epoch_end(self, outputs) -> Any:
    #     return torch.stack([x["y"] for x in outputs]).mean()

    def configure_optimizers(self) -> tuple[list[Any], list[Any]]:
        optimizer = torch.optim.SGD(self.layer.parameters(), lr=0.1)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

In [7]:
from lit_mlflow import MlFlowAutoCallback, DbxMLFlowLogger

ModuleNotFoundError : No module named 'lit_mlflow'

Error: [0;31m---------------------------------------------------------------------------[0m
[0;31mModuleNotFoundError[0m                       Traceback (most recent call last)
File [0;32m~/.ipykernel/4410/command--1-2891841985:1[0m
[0;32m----> 1[0m [38;5;28;01mfrom[39;00m [38;5;21;01mlit_mlflow[39;00m [38;5;28;01mimport[39;00m MlFlowAutoCallback, DbxMLFlowLogger

[0;31mModuleNotFoundError[0m: No module named 'lit_mlflow'

In [8]:
from lightning.pytorch.callbacks import DeviceStatsMonitor

In [ ]:
from lightning.pytorch.callbacks.progress import ProgressBar, RichProgressBar, TQDMProgressBar

In [ ]:
import mlflow

mlflow.enable_system_metrics_logging()

In [9]:
dm = RandomDataModule()

model = BoringModel()

# Initialize a trainer
trainer = L.Trainer(
    limit_train_batches=1000,
    limit_val_batches=100,
    limit_test_batches=10,
    num_sanity_val_steps=0,
    max_epochs=50,
    enable_model_summary=False,
    logger=DbxMLFlowLogger(),
    callbacks=[RichProgressBar(), MlFlowAutoCallback()],
)

# Train the model ⚡
trainer.fit(model, datamodule=dm)

trainer.test(datamodule=dm, ckpt_path="best")

NameError : name 'DbxMLFlowLogger' is not defined

Error: [0;31m---------------------------------------------------------------------------[0m
[0;31mNameError[0m                                 Traceback (most recent call last)
File [0;32m~/.ipykernel/4410/command--1-1698900115:13[0m
[1;32m      3[0m model [38;5;241m=[39m BoringModel()
[1;32m      5[0m [38;5;66;03m# Initialize a trainer[39;00m
[1;32m      6[0m trainer [38;5;241m=[39m L[38;5;241m.[39mTrainer(
[1;32m      7[0m     limit_train_batches[38;5;241m=[39m[38;5;241m100[39m,
[1;32m      8[0m     limit_val_batches[38;5;241m=[39m[38;5;241m10[39m,
[1;32m      9[0m     limit_test_batches[38;5;241m=[39m[38;5;241m10[39m,
[1;32m     10[0m     num_sanity_val_steps[38;5;241m=[39m[38;5;241m0[39m,
[1;32m     11[0m     max_epochs[38;5;241m=[39m[38;5;241m10[39m,
[1;32m     12[0m     enable_model_summary[38;5;241m=[39m[38;5;28;01mFalse[39;00m,
[0;32m---> 13[0m     logger[38;5;241m=[39mDbxMLFlowLogger(),
[1;32m     14[0m     callback